In [22]:
import torch
from collections import namedtuple, deque
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from gym.spaces import Box
import matplotlib.pyplot as plt

import numpy as np


In [13]:
# Replay Memory
Transition = namedtuple('Transition',
                        ('state', 'action', 'next_state', 'reward', 'done'))


class ReplayMemory(object):

    def __init__(self, capacity):
        self.memory = deque([], maxlen=capacity)

    def push(self, *args):
        """Save a transition"""
        self.memory.append(Transition(*args))

    def sample(self, batch_size):
        transitions = random.sample(self.memory, batch_size)
        batch = Transition(*zip(*transitions))
        state_batch = np.vstack(batch.state)
        action_batch = np.vstack(batch.action)
        reward_batch = np.vstack(batch.reward)
        next_state_batch = np.vstack(batch.next_state)
        done_batch = np.vstack(batch.done)
        return state_batch, action_batch, next_state_batch, reward_batch, done_batch

    def __len__(self):
        return len(self.memory)

In [14]:
def rbf_function_on_action(centroid_locations, action, beta):
    '''
    centroid_locations: Tensor [batch x num_centroids (N) x a_dim (action_size)]
    action_set: Tensor [batch x a_dim (action_size)]
    beta: float
        - Parameter for RBF function

    Description: Computes the RBF function given centroid_locations and one action
    '''
    assert len(centroid_locations.shape) == 3, "Must pass tensor with shape: [batch x N x a_dim]"
    assert len(action.shape) == 2, "Must pass tensor with shape: [batch x a_dim]"

    diff_norm = centroid_locations - action.unsqueeze(dim=1).expand_as(centroid_locations)
    diff_norm = diff_norm**2
    diff_norm = torch.sum(diff_norm, dim=2)
    diff_norm = torch.sqrt(diff_norm + 1e-7)
    diff_norm = diff_norm * beta * -1
    weights = F.softmax(diff_norm, dim=1)  # batch x N
    return weights


def rbf_function(centroid_locations, action_set, beta):
	'''
	centroid_locations: Tensor [batch x num_centroids (N) x a_dim (action_size)]
	action_set: Tensor [batch x num_act x a_dim (action_size)]
		- Note: pass in num_act = 1 if you want a single action evaluated
	beta: float
		- Parameter for RBF function

	Description: Computes the RBF function given centroid_locations and some actions
	'''
	assert len(centroid_locations.shape) == 3, "Must pass tensor with shape: [batch x N x a_dim]"
	assert len(action_set.shape) == 3, "Must pass tensor with shape: [batch x num_act x a_dim]"

	diff_norm = torch.cdist(centroid_locations, action_set, p=2)  # batch x N x num_act
	diff_norm = diff_norm * beta * -1
	weights = F.softmax(diff_norm, dim=2)  # batch x N x num_act
	return weights

In [15]:
class Reshape(torch.nn.Module):
	"""
	Description:
		Module that returns a view of the input which has a different size
	Parameters:
		- args : Int...
			The desired size
	"""
	def __init__(self, *args):
		super().__init__()
		self.shape = args

	def __repr__(self):
		s = self.__class__.__name__
		s += '{}'.format(self.shape)
		return s

	def forward(self, x):
		return x.view(*self.shape)

def sync_networks(target, online, alpha, copy=False):
	if copy == True:
		for online_param, target_param in zip(online.parameters(), target.parameters()):
			target_param.data.copy_(online_param.data)
	elif copy == False:
		for online_param, target_param in zip(online.parameters(), target.parameters()):
			target_param.data.copy_(alpha * online_param.data +
			                        (1 - alpha) * target_param.data)

In [16]:
class Net(nn.Module):
    def __init__(self, params, env, state_size, action_size, device):
        super(Net, self).__init__()
        self.env = env
        self.device = device
        self.params = params
        self.N = self.params['num_points']
        self.max_a = self.env.action_space.high[0]
        self.beta = self.params['temperature']

        self.buffer_object = ReplayMemory(
            capacity=self.params['max_buffer_size'])

        self.state_size, self.action_size = state_size, action_size

        self.value_module = nn.Sequential(
            nn.Linear(self.state_size, self.params['layer_size']),
            # Is ReLU a natural choice?
            nn.ReLU(),
            nn.Linear(self.params['layer_size'], self.params['layer_size']),
            nn.ReLU(),
            nn.Linear(self.params['layer_size'], self.params['layer_size']),
            nn.ReLU(),
            # Maybe a linear layer isn't ideal here
            nn.Linear(self.params['layer_size'], self.N),
        )

        if self.params['num_layers_action_side'] == 1:
            self.location_module = nn.Sequential(
                nn.Linear(self.state_size, self.params['layer_size_action_side']),
                nn.Dropout(p=self.params['dropout_rate']),
                nn.ReLU(),
                nn.Linear(self.params['layer_size_action_side'],
                            self.action_size * self.N),
                Reshape(-1, self.N, self.action_size),
                # Change from tanh to sigmoid for custom action space
                nn.Sigmoid(),
            )
        elif self.params['num_layers_action_side'] == 2:
            self.location_module = nn.Sequential(
                nn.Linear(self.state_size, self.params['layer_size_action_side']),
                nn.Dropout(p=self.params['dropout_rate']),
                nn.ReLU(),
                nn.Linear(self.params['layer_size_action_side'],
                          self.params['layer_size_action_side']),
                nn.Dropout(p=self.params['dropout_rate']),
                nn.ReLU(),
                nn.Linear(self.params['layer_size_action_side'],
                          self.action_size * self.N),
                Reshape(-1, self.N, self.action_size),
                nn.Sigmoid(),
            )
        torch.nn.init.xavier_uniform_(self.location_module[0].weight)
        torch.nn.init.zeros_(self.location_module[0].bias)

        self.location_module[3].weight.data.uniform_(-.1, .1)
        self.location_module[3].bias.data.uniform_(-1., 1.)

        self.criterion = nn.SmoothL1Loss()

        self.params_dic = [{
            'params': self.value_module.parameters(), 'lr': self.params['learning_rate']
        },
                            {
                                'params': self.location_module.parameters(),
                                'lr': self.params['learning_rate_location_side']
                            }]
        try:
            if self.params['optimizer'] == 'RMSprop':
                self.optimizer = optim.RMSprop(self.params_dic)
            elif self.params['optimizer'] == 'Adam':
                self.optimizer = optim.Adam(self.params_dic)
            else:
                print('unknown optimizer ....')
        except:
            print("no optimizer specified ... ")
        self.to(self.device)

    def get_centroid_values(self, s):
        '''
        given a batch of s, get all centroid values, [batch x N]
        '''
        centroid_values = self.value_module(s)
        return centroid_values

    def get_centroid_locations(self, s):
        '''
        given a batch of s, get all centroid_locations, [batch x N x a_dim]
        '''
        centroid_locations = self.max_a * self.location_module(s)
        return centroid_locations

    def get_best_qvalue_and_action(self, s):
        '''
        given a batch of states s, return Q(s,a), max_{a} ([batch x 1], [batch x a_dim])
        '''
        all_centroids = self.get_centroid_locations(s)
        values = self.get_centroid_values(s)
        weights = rbf_function(all_centroids, all_centroids, self.beta)  # [batch x N x N]
        allq = torch.bmm(weights, values.unsqueeze(2)).squeeze(2)  # bs x num_centroids
        # a -> all_centroids[idx] such that idx is max(dim=1) in allq
        # a = torch.gather(all_centroids, dim=1, index=indices)
        # (dim: bs x 1, dim: bs x action_dim)
        best, indices = allq.max(dim=1)
        if s.shape[0] == 1:
            index_star = indices.item()
            a = all_centroids[0, index_star]
            return best, a
        else:
            return best, None

    def forward(self, s, a):
        '''
        given a batch of s, a, compute Q(s,a) [batch x 1]
        '''
        centroid_values = self.get_centroid_values(s)  # [batch_dim x N]
        centroid_locations = self.get_centroid_locations(s)
        # [batch x N]
        centroid_weights = rbf_function_on_action(centroid_locations, a, self.beta)
        output = torch.mul(centroid_weights, centroid_values)  # [batch x N]
        output = output.sum(1, keepdim=True)  # [batch x 1]
        return output

    def e_greedy_policy(self, s, episode, train_or_test):
        '''
        Given state s, at episode, take random action with p=eps if training
        Note - epsilon is determined by episode
        '''
        epsilon = 1.0 / np.power(episode, 1.0 / self.params['policy_parameter'])
        if train_or_test == 'train' and random.random() < epsilon:
            a = self.env.action_space.sample()
            return a.tolist()
        else:
            self.eval()
            s_matrix = np.array(s).reshape(1, self.state_size)
            with torch.no_grad():
                s = torch.from_numpy(s_matrix).float().to(self.device)
                _, a = self.get_best_qvalue_and_action(s)
                a = a.cpu().numpy()
            self.train()
            return a

    def update(self, target_Q):
        if len(self.buffer_object) < self.params['batch_size']:
            return 0
        s_matrix, a_matrix, sp_matrix, r_matrix, d_matrix = self.buffer_object.sample(self.params['batch_size'])
        #r_matrix = np.clip(r_matrix,
        #                    a_min=-self.params['reward_clip'],
        #                    a_max=self.params['reward_clip'])

        s_matrix = torch.from_numpy(s_matrix).float().to(self.device)
        a_matrix = torch.from_numpy(a_matrix).float().to(self.device)
        r_matrix = torch.from_numpy(r_matrix).float().to(self.device)
        sp_matrix = torch.from_numpy(sp_matrix).float().to(self.device)
        d_matrix = torch.from_numpy(d_matrix).float().to(self.device)

        Q_star, _ = target_Q.get_best_qvalue_and_action(sp_matrix)
        Q_star = Q_star.reshape((self.params['batch_size'], -1))
        with torch.no_grad():
            y = r_matrix + self.params['gamma'] * Q_star
        y_hat = self.forward(s_matrix, a_matrix)
        loss = self.criterion(y_hat, y)
        self.zero_grad()
        loss.backward()
        self.optimizer.step()
        #self.zero_grad()
        sync_networks(
            target=target_Q,
            online=self,
            alpha=self.params['target_network_learning_rate'],
            copy=False)
        return loss.cpu().data.numpy()

In [ ]:
params = {'env_name': 'solow',
          'env': env,
          'max_episode': 300,
          'num_layers': 2,
          'layer_size': 256,
          'num_layers_action_side': 1,
          'layer_size_action_side': 256,
          'learning_rate': 0.00025,
          'learning_rate_location_side': 2.5e-05,
          'target_network_learning_rate': 0.005,
          'max_buffer_size': 50000,
          'gamma': env.β,
          'batch_size': 256,
          'num_points': 100,
          'temperature': 1,
          'policy_parameter': 1.5, # Determines how fast exploration decays
          'norm_smoothing': 1e-05,
          'updates_per_episode': 500,
          'burn_steps': 15,
          'dropout_rate': 0,
          'optimizer': 'Adam',
          'policy_type': 'e_greedy',
          'seed_number': 2056}


In [20]:
class GrowthModel:
    def __init__(self,
                u,            # utility function
                f,            # production function
                δ=0.1,        # capital depreciation
                β=0.96,       # discount factor
                μ=0,          # shock location parameter
                k_max=4,      # max capital
                s=0.1,        # shock scale parameter
                c_high=1,     # consumption bound
                ):

        self.u, self.f, self.β, self.μ, self.s, self.k_max, self.δ = u, f, β, μ, s, k_max, δ
        # Use gym action space to accomodate continuous actions
        self.action_space = Box(low=0, high=c_high, shape=(1, ), dtype=np.float32)
        # What if, isntead of using percentage, we use number (bounding max consumption by 3)
        # self.action_space = Box(low=0, high=3, shape=(1, ), dtype=np.float32)

    def reset(self):
        """
        Resets the current_state variable.
        """
        k_init = np.random.beta(5, 5)
        shock = self.μ + self.s*np.random.randn()
        self.current_state = np.array([k_init, shock])
        self.count = 0
        return self.current_state

    def step(self, action: float):
        """
        Takes a step in the environment by sampling from the
        transition matrix self.T given an action and the current_state and return the index of the next state.
        It returns a tuple of (next_state, reward, terminal, truncation, flags).
        """
        assert self.current_state is not None, "State has not been reset"
        terminal, truncation = False, False
        # Current period states
        k, shock = self.current_state
        # Compute macro variables
        y = np.exp(shock)*self.f(k)
        c = action[0]*y
        reward = self.u(c)
        # Next period states
        next_k = y - c + (1 - self.δ)*k
        shock = self.μ + self.s*np.random.randn()
        self.current_state = np.array([next_k, shock])
        self.count += 1
        if self.count >= 1000:
            truncation = True
        if next_k == 0:
            terminal = True
        return self.current_state, reward, terminal, truncation

    @property
    def features_size(self):
        return self.phi(0).shape[0]
α = 0.4
def fcd(k):
    return np.nan_to_num(np.power(k, α), nan=-np.inf)

env = GrowthModel(u=np.log, f=fcd, δ=0.1)
env.reset()

array([0.44097944, 0.22297295])

In [55]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("Running on the GPU")
else:
    device = torch.device("cpu")
    print("Running on the CPU")  

params = {'env_name': 'solow',
          'env': env,
          'max_episode': 300,
          'num_layers': 2,
          'layer_size': 256,
          'num_layers_action_side': 1,
          'layer_size_action_side': 256,
          'learning_rate': 0.00025,
          'learning_rate_location_side': 2.5e-05,
          'target_network_learning_rate': 0.005,
          'max_buffer_size': 50000,
          'gamma': env.β,
          'batch_size': 256,
          'num_points': 100,
          'temperature': 1,
          'policy_parameter': 1.5, # Determines how fast exploration decays
          'norm_smoothing': 1e-05,
          'updates_per_episode': 500,
          'burn_steps': 15,
          'dropout_rate': 0,
          'optimizer': 'Adam',
          'policy_type': 'e_greedy',
          'seed_number': 2056}
s0 = env.reset()
ks = np.linspace(0.1, 0.5, 400)
s = np.vstack([ks, np.zeros_like(ks)]).T
vstar = v_star(ks, α, env.β, env.μ)

for n in np.arange(start=50, stop=401, step=50):
    params['num_points'] = n
    Q_object_target = Net(params,
                            env,
                            state_size=len(s0),
                            action_size=len(env.action_space.low),
                            device=device)
    Q_object_target.load_state_dict(torch.load(f'models/model_{n}_1_299', map_location=torch.device('cpu')))

    with torch.no_grad():
        ks_tensor = torch.tensor(s, device=device, dtype=torch.float)
        v, _ = Q_object_target.get_best_qvalue_and_action(ks_tensor)
        v = v.detach().cpu().numpy()
    actions = []
    for i in ks:
        s = torch.tensor([i, 0], device=device, dtype=torch.float).reshape(1, 2)
        _, a = Q_object_target.get_best_qvalue_and_action(s)
        actions.append(a.cpu().detach().numpy())
    actions = np.array(actions).reshape(ks.shape)
    print(f"n = {n}, value SSE = {np.mean((v - vstar)**2)}, action SSE = {np.sum((actions - (1 - α * env.β))**2)}")
    

Running on the CPU
n = 50, value SSE = 0.0002969293106994044, action SSE = 0.21659845113754272


/var/folders/8d/sm7kcrm54cd6hg3rnqsh6sxm0000gp/T/ipykernel_1384/607774263.py:46: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  ks_tensor = torch.tensor(s, device=device, dtype=torch.float)


n = 100, value SSE = 0.21230601990407355, action SSE = 0.3093411922454834
n = 150, value SSE = 0.23854210510424165, action SSE = 0.013707506470382214
n = 200, value SSE = 0.24120404772592116, action SSE = 0.05019858479499817
n = 250, value SSE = 0.25456966184704716, action SSE = 0.026028042659163475
n = 300, value SSE = 0.22402535847422606, action SSE = 0.0440058708190918
n = 350, value SSE = 0.2617604182302153, action SSE = 0.02421414665877819
n = 400, value SSE = 0.20410422349822388, action SSE = 0.043907564133405685


In [29]:
def v_star(k, α, β, μ):
    """
    True value function
    """
    c1 = np.log(1 - α * β) / (1 - β)
    c2 = (μ + α * np.log(α * β)) / (1 - α)
    c3 = 1 / (1 - β)
    c4 = 1 / (1 - α * β)
    return c1 + c2 * (c3 - c4) + c4 * α * np.log(k)

In [37]:
ks = np.linspace(0.1, 0.5, 400)
s = np.vstack([ks, np.zeros_like(ks)]).T
with torch.no_grad():
    ks_tensor = torch.tensor(s, device=device, dtype=torch.float)
    v, _ = Q_object_target.get_best_qvalue_and_action(ks_tensor)
    v = v.detach().cpu().numpy()
vstar = v_star(ks, α, env.β, env.μ)
np.sum((v - vstar) ** 2)

0.25022193171840845

In [52]:
actions = []
for i in ks:
    s = torch.tensor([i, 0], device=device, dtype=torch.float).reshape(1, 2)
    _, a = Q_object_target.get_best_qvalue_and_action(s)
    actions.append(a.cpu().detach().numpy())
np.sum((np.array(actions).reshape(ks.shape) - (1 - α * env.β)) ** 2)

0.024214147